# Perceptual Losses for styled coco dataset

In this notebook, we generate the perceptual losses for the coco and styled-coco images in an offline manner, so that later we can load in the dataloder as metas for training the faster-rcnn

In [46]:
import os, pdb
import sys
import json
import imageio
import tqdm

import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision

sys.path.append("..")

from data import StyledCoco
from CONFIG import CONFIG
from lib.loss import VGGPerceptualLoss

In [2]:
%reload_ext autoreload
%autoreload 2

In [4]:
data_path = CONFIG["paths"]["data_path"]
coco_data_path = CONFIG["paths"]["coco_data_path"]

train_images_path = os.path.join(data_path, "images", "train")
train_original_imgs_path = os.path.join(coco_data_path, "original_images", "train2017")

valid_images_path = os.path.join(data_path, "images", "valid")
valid_original_imgs_path = os.path.join(coco_data_path, "original_images", "val2017")

labels_path = os.path.join(data_path, "annotations")
labels_file = os.path.join(labels_path, "person_keypoints_train.json")

In [ ]:
original_list = os.listdir(train_original_imgs_path)
original_list = [i for i in original_list if '.ipynb_checkpoints' != i]
                                              
styled_list = os.listdir(train_images_path)    
styled_list = [i for i in styled_list if '.ipynb_checkpoints' != i]

In [47]:
count = 0                                     
vgg_perceptual_loss = VGGPerceptualLoss()

for o_name in tqdm.tqdm(original_list):            
    count += 1                                
    if count % 1000 == 0:                     
        print ('{} images done'.format(str(count)))
        
    for i in styled_list:                     
        if o_name.split('.')[0] in i:         
            s_name = i                        
                                              
    o_path = os.path.join(train_original_imgs_path, o_name)
    s_path = os.path.join(train_images_path, s_name) 
    
    o_image = imageio.imread(o_path)
    s_image = imageio.imread(s_path)
                                     
    #show_images_in_parallel(o_image, s_image)
    
    o_array = o_image.transpose(2,0,1)/255.0
    s_array = s_image.transpose(2,0,1)/255.0
    
    o_tensor = torch.Tensor(o_array).unsqueeze(0)
    s_tensor = torch.Tensor(s_array).unsqueeze(0)
    print (o_tensor.shape, s_tensor.shape)
    
    vgg_loss_value = vgg_perceptual_loss.forward(o_tensor, s_tensor)
    print (vgg_loss_value)    
    break

  0%|          | 0/118287 [00:00<?, ?it/s]

torch.Size([1, 3, 480, 640]) torch.Size([1, 3, 480, 640])
tensor(4.5574, grad_fn=<AddBackward0>)


In [19]:
def show_images_in_parallel(o_image, s_image):
    fig, ax = plt.subplots(1,2)
    fig.set_size_inches(11,5)

    ax[0].imshow(s_image)
    ax[0].set_title(f"Styled Image {s_name}")
    ax[0].set_axis_off()
    
    ax[1].set_axis_off()
    ax[1].imshow(o_image)
    ax[1].set_title(f"Original Image {o_name}")
    plt.tight_layout()

(480, 640, 3)

In [25]:
temp = o_image.transpose(2,0,1)

In [30]:
import torch
torch.Tensor(temp).unsqueeze(0).shape

torch.Size([1, 3, 480, 640])